In [1]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

Using TensorFlow backend.


In [2]:
embed_size = 50 # how big is each word vector
max_features = 20000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a comment to use

In [3]:
train = pd.read_csv("train1.csv")
test = pd.read_csv("test.csv")

In [4]:
path='C:\\Users\\Manjunath Purma\\Downloads\\text mining\\cute 04\\'
EMBEDDING_FILE=f'{path}glove6b50d\\glove.6B.50d.txt'


In [5]:
list_sentences_train = train["converse"].fillna("_na_").values
list_classes = ["APPOINTMENTS ","ASK_A_DOCTOR", "JUNK", "LAB", "MISCELLANEOUS", "PRESCRIPTION"]
y = train[list_classes].values
list_sentences_test = test["converse"].fillna("_na_").values

In [6]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [7]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE,encoding='utf-8'))

In [8]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std

(0.020940498, 0.6441043)

In [9]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [10]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
model.fit(X_t, y, batch_size=32, epochs=10, validation_split=0.1);

Train on 41241 samples, validate on 4583 samples
Epoch 1/10
41241/41241 [==============================] - 472s - loss: 0.2347 - acc: 0.9028 - val_loss: 0.1818 - val_acc: 0.9224
Epoch 2/10
41241/41241 [==============================] - 440s - loss: 0.1752 - acc: 0.9279 - val_loss: 0.1733 - val_acc: 0.9278
Epoch 3/10
41241/41241 [==============================] - 428s - loss: 0.1613 - acc: 0.9342 - val_loss: 0.1678 - val_acc: 0.9296
Epoch 4/10
41241/41241 [==============================] - 431s - loss: 0.1510 - acc: 0.9384 - val_loss: 0.1668 - val_acc: 0.9288
Epoch 5/10
41241/41241 [==============================] - 438s - loss: 0.1425 - acc: 0.9426 - val_loss: 0.1709 - val_acc: 0.9294
Epoch 6/10
41241/41241 [==============================] - 438s - loss: 0.1342 - acc: 0.9469 - val_loss: 0.1704 - val_acc: 0.9292
Epoch 7/10
41241/41241 [==============================] - 437s - loss: 0.1257 - acc: 0.9506 - val_loss: 0.1778 - val_acc: 0.9249
Epoch 8/10
41241/41241 [========================

In [12]:
y_test = model.predict([X_te], batch_size=1024, verbose=1)

11456/11456 [==============================] - 29s    


In [13]:
#y_pred_class = model.predict_classes(X_te)
y_test

array([[  1.18897983e-03,   9.04069399e-04,   4.04458442e-05,
          1.86832607e-04,   9.94617760e-01,   3.59877769e-04],
       [  3.31805527e-01,   6.80973008e-03,   3.63130937e-04,
          2.85223272e-04,   6.40478075e-01,   1.83756314e-02],
       [  6.56342553e-03,   1.97325628e-02,   1.90916609e-07,
          1.35611452e-03,   1.14390045e-01,   8.83680522e-01],
       ..., 
       [  8.09499715e-03,   8.12515058e-03,   2.25259282e-04,
          4.15296759e-04,   9.82283056e-01,   3.04060261e-04],
       [  9.83239353e-01,   6.77874731e-03,   3.37692218e-05,
          5.08715282e-04,   3.89011367e-03,   1.92561012e-04],
       [  3.37180332e-03,   9.24005926e-01,   2.28237274e-09,
          9.25716886e-04,   3.43167270e-03,   6.42103329e-02]], dtype=float32)

In [14]:
y_pred_class=np.argmax(y_test,axis=1)



In [15]:
y_pred_class

array([4, 4, 5, ..., 4, 0, 1], dtype=int64)

In [16]:
np.savetxt("foo3.csv", y_pred_class, delimiter=",")